In [38]:
import os
import string
import annoy
import codecs

from pymorphy3 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее. Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста

In [39]:
question = None
written = False

#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with codecs.open("./data/prepared_answers.txt","w", "utf-8") as fout:
    with codecs.open("./data/answers.txt", "r", "utf-8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

Теперь нам нужно предобработать текст, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [40]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [41]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with codecs.open("./data/answers.txt", "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

In [42]:
sentences

[[],
 [],
 ['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['хомячок'],
 ['мужик', 'йопарить', 'собачка', '50', 'кошка'],
 ['общение'],
 ['паучок'],
 ['пол', 'памытьbr', 'таг', 'тип', 'каво'],
 ['вообще', 'пообщаться'],
 ['советовать', 'сися', 'завести'],
 ['слава', 'бог', 'мужик', 'завести', 'советовать', 'сватать', 'сватать'],
 ['советовать', 'завести', 'любовник', 'муж', 'кошка', 'brприветик'],
 [],
 ['парень',
  'относиться',
  'цветной',
  'линза',
  'девушка',
  'зелёный',
  'глаз',
  'голубой'],
 ['вобщий', 'прикалывать', 'тема'],
 ['этобыть',
  'редкость',
  'забавно',
  'знать',
  'фальшивка',
  'прикольно',
  'силиконовый',
  'сиська',
  'налепить',
  'синтетический',
  'волос'],
 [],
 ['делать', 'найти', '2', 'миллион', 'рубль'],
 ['счастие',
  'свалиться',
  'хороший',
  'пойти',
  'милиция',
  'заявить',
  'находка',
  'деньга',
  'тероть',
  'самый',
  'интересный',
  'неприменный',
  'искать',
  'поверьте',
  'найти',
  'видеть',
  'подобный',
  'нарват

In [43]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("./data/w2v_model")

Теперь нам нужно сложить в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, считаем, что вектор предложения - сумма word2vecов слов, которые входят в него (конечно же усредненная)

In [44]:
index = annoy.AnnoyIndex(100, 'angular')  # Создаем объект AnnoyIndex с размерностью 100 и метрикой "angular"

index_map = {}  # Создаем пустой словарь для отображения индексов на строки
counter = 0  # Устанавливаем счетчик на 0

with codecs.open("./data/prepared_answers.txt", "r", "utf-8") as f:  # Открываем файл "prepared_answers.txt" для чтения
    for line in tqdm(f):  # Итерируем по строкам файла
        n_w2v = 0  # Устанавливаем счетчик слов, для которых есть векторы Word2Vec, в 0
        spls = line.split("\t")  # Разделяем строку на элементы по табуляции
        index_map[counter] = spls[1]  # Сопоставляем индекс с соответствующей строкой из файла
        question = preprocess_txt(spls[0])  # Предобрабатываем текст вопроса
        vector = np.zeros(100)  # Создаем нулевой вектор размерностью 100
        for word in question:  # Итерируем по словам в вопросе
            if word in model.wv:  # Проверяем, есть ли у слова вектор в Word2Vec модели
                vector += model.wv[word]  # Добавляем вектор слова к общему вектору вопроса
                n_w2v += 1  # Увеличиваем счетчик слов с векторами Word2Vec
        if n_w2v > 0:  # Проверяем, что были найдены векторы для слов в вопросе
            vector = vector / n_w2v  # Нормируем вектор вопроса
        index.add_item(counter, vector)  # Добавляем вектор вопроса в AnnoyIndex с соответствующим индексом
        counter += 1  # Увеличиваем счетчик

index.build(10)  # Строим дерево в AnnoyIndex с 10 деревьями
index.save('./data/speaker.ann')  # Сохраняем индекс в файл "speaker.ann"


0it [00:00, ?it/s]

True

In [11]:
'''index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with codecs.open("./data/prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('./data/speaker.ann')'''

0it [00:00, ?it/s]

True

Теперь остается реализовать метод, который получит на вход вопрос и найдет ответ к нему!
Мы препроцессим вопрос, находим ближайший вопрос и выбираем ответ на ближайший вопрос.

In [45]:
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]

In [46]:
test = "Что за бред?"

In [47]:
print(find_answer(test))

открой солнечного бреда, у него каждый вопрос пример http://otvet.mail.ru/mail/mr.suddenly/. 



----